## Scenario 2: Train and deploy with MLFlow and AML


1)	Train baseline model and log / autolog with MLFlow and submit job with AML CLI/MLFLow CLI

2)	Test model locally with v2 CLI and manually validate results

3)	Register the model from the run 

4)	{Optional} Change the model stage to “Production” and discuss with team before deploying to production (**NOTE:** Not fully integrate in AML UI yet)
    
5)	After user is satisfied with the model, deploy model on AML to an endpoint and use endpoint to predict all the result from a dataset. (**NOTE: ** TensorSpec not supported yet)

## Move to AML by setting the tracking URI in the backend (not in my training code), and using MLFlow CLI. 

In [ ]:
!az extension add -n ml -y 

In [ ]:
!az login

In [ ]:
import subprocess

#Get MLFLow UI through the Azure ML CLI v2 and convert to string
MLFLOW_TRACKING_URI = subprocess.run(["az", "ml", "workspace", "show", "--query", "mlflow_tracking_uri", "-o", "tsv"], stdout=subprocess.PIPE, text=True)
MLFLOW_TRACKING_URI = str(MLFLOW_TRACKING_URI.stdout).strip()

## Make sure the MLFLow URI looks something like this: 
## azureml://westus.api.azureml.ms/mlflow/v1.0/subscriptions/<Sub-ID>/resourceGroups/<RG>/providers/Microsoft.MachineLearningServices/workspaces/<WS>
print("MLFlow Tracking URI:", MLFLOW_TRACKING_URI)

In [ ]:
## Set the MLFLOW TRACKING URI
import mlflow
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

## Train baseline model and log / autolog with MLFlow and submit job with MLFLow CLI

In [ ]:
backend_config = {"USE_CONDA": False}

In [ ]:
import mlflow
local_env_run = mlflow.projects.run(uri="simple_project", 
                                    parameters={"alpha":0.2},
                                    experiment_name="Scenario2_project",
                                    backend = "azureml",
                                    use_conda=True,
                                    backend_config = backend_config, 
                                    )

### Download or retrieve the model from the run for testing

In [ ]:
from mlflow.entities import ViewType
experiment_name="Scenario2_project"
current_experiment=mlflow.get_experiment_by_name(experiment_name)
runs = mlflow.search_runs(experiment_ids=current_experiment.experiment_id, run_view_type=ViewType.ALL)

In [ ]:
last_run_id = runs.tail(1)["run_id"].tolist()[0]
last_run_id

### Download and Load Test Data from JSON

In [ ]:
from mlflow.tracking.client import MlflowClient
client = MlflowClient()
client.download_artifacts(last_run_id,"model/input_example.json",".")

In [ ]:
import json

with open('model/input_example.json') as f:
   sample_data = json.load(f)

#columns = ['age', 'gender', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6']
print(sample_data)

In [ ]:
model_path = "model"
artifact_uri = "runs:/{}/{}".format(last_run_id,model_path)
model = mlflow.sklearn.load_model(artifact_uri)

In [ ]:
model.predict(sample_data["inputs"])

## Register Model with MLFLow

In [ ]:
mlflow.register_model(artifact_uri,"scenario2model")

## (Optional) Transistion Model to Production Stage

In [ ]:
client = MlflowClient()
client.transition_model_version_stage(
    name="scenario2model",
    version=1,
    stage="Production"
)

## List Model details

In [ ]:
from pprint import pprint
client = MlflowClient()
for mv in client.search_model_versions("name='scenario2model'"):
    pprint(dict(mv), indent=4)

## Deploy to AzureML's MIR with MLFLow
As of Dec 2021, Not Support:
- TensorSpec for Deployment in AML is not fully supported
- Endpoints are not fully supported with MLFlow deployment, there is a 1:1 relationship between deployment and endpoint. MLFlow needs to support the ability to deploy multiple models to the 1 endpoint

In [ ]:
from mlflow.deployments import get_deploy_client
import mlflow
import mlflow.sklearn

# set the tracking uri as the deployment client
client = get_deploy_client(mlflow.get_tracking_uri())

# set the model path 
# model_path = "model"
# run_id= "13c5faef-788f-439d-ba6c-cb8d280e708d"

# Retrieve model from registry
model_name = "scenario2model"
model_version = 1
model_uri = 'models:/{}/{}'.format(model_name, model_version)

# define the model path and the name is the service name
# the model gets registered automatically and a name is autogenerated using the "name" parameter below 
# set the deployment config
deploy_path = "deployment_config_v2.json"
test_config = {'deploy-config-file': deploy_path}

client.create_deployment(model_uri=model_uri,
                         config=test_config,
                         name="mlflowscenario3")

## Test endpoint with MLFlow

Find code snippet below, in the Endpoint UI in AzureML and navigate to the Consume Tab for the Endpoint you just deployed

In [ ]:
import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

# Request data goes here
data = sample_data

body = str.encode(json.dumps(data))

url = 'https://mlflowscenario2.westus.inference.ml.azure.com/score'
api_key = '' # Replace this with the API key for the web service
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(json.loads(error.read().decode("utf8", 'ignore')))